# Curvas mexicanas
El propósito de este documento es el de mostar los pasos para evaluar las cuvas de TIIE28 y de FTIIE respectivamente.

Se explicará el proceso que se usa para valuar estas curvas por medio de `QuantLib`, librería que se usa en la valuación y creación de instrumentos financieros. Se hará un ejemplo con las curvas del 19 de febrero de 2025 de acuerdo al modelo de CME. Este modelo se llama Boostrapping paramétrico, pues usa métodos numéricos para el ajuste de las curvas.

La curva TIIE28, FTIIE, y SOFR son curvas que se usan para calcular las tasas de los cupones variables de los swaps, ietras que la de curva de Descuento es para descontar los flujos del Swap. En el caso de la SOFR, se usa la mism curva como curva de descuento, pues es considerada una curva libre de riesgo (al ser muy líquida).

Los pasos a seguir para la creación de cada curva son los siguientes: (más info en el siguiente [paper](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=2219548&download=yes) )
- Definir los insumos necesarios 
- Definir objetos para cada instrumento necesario para el ajuste de curvas
- Hacer el bootstrapping 

Vamos a definir las curvas en el siguiente orden:  
1. Curva de TIIE28
2. Curva de FTIIE
3. Curva de Descuento MXN
4. Curva de SOFR

## Curva de TIIE28  

### Definción de Insumos
Siguiendo los pasos antes mencionados, primero necesitamos definir los insumos que se necesitan para esta curva en específico. De acuerdo al modelo de CME, la curva de TIIE28 usa las tasas par de cierre para los siguientes nodos (expresados en 28 días):

- 1: 1 mes
- 3: 3 meses
- 6: 6 meses
- 9: 9 meses
- 13: 1 año
- 26: 2 años
- 39: 3 años
- 52: 4 años
- 65: 5 años
- 91: 7 años
- 130: 10 años
- 195: 15 años
- 260: 20 años
- 390: 30 años

In [ ]:
# Librerías a utilizar
import numpy as np
import pandas as pd
import QuantLib as ql # versión 1.38 o superior

# Definición de insumos
# L es un mes de 28 días (mes lunar)

t_1L = 9.7638
t_3L = 9.643802
t_6L = 9.293893
t_9L = 9.066304
t_13L = 8.895611
t_26L = 8.719446
t_39L = 8.713905
t_52L = 8.764028
t_65L = 8.814004
t_91L = 8.938174
t_130L = 9.094733
t_195L = 9.276024
t_260L = 9.293577
t_390L = 9.2277

# Definición de tenors
tenors_tiie28 = [
    1, # 1 mes
    3, # 3 meses
    6, # 6 meses
    9, # 9 meses
    13, # 13 meses
    26, # 26 meses
    39, # 39 meses
    52, # 52 meses
    65, # 65 meses
    91, # 91 meses
    130, # 130 meses
    195, # 195 meses
    260, # 260 meses
    390  # 390 meses
]

# Definición de tasas
tasas_tiie28 = [
    t_1L,
    t_3L,
    t_6L,
    t_9L,
    t_13L,
    t_26L,
    t_39L,
    t_52L,
    t_65L,
    t_91L,
    t_130L,
    t_195L,
    t_260L,
    t_390L
]

### Definición de Objetos
En la librería QuantLib, se necesitan definir objetos tipo `helpers` para cada instrumento de la curva. En el caso de la curva de TIIE28, los ojetos que usaremos serán los siguientes:
- `ql.DepositRatehelper`: objeto para tasas cero (la tasa par de un mes se puede definir como una tasa cero)
- `ql.SwapRateHelper`: objeto para tasas par de swaps

Además de estos objetos, tambien es necesario definir el objeto `ql.IborIndex`, el cual contiene las especificacione sde la curva que se esta ajustando (tipo de conteo de días, manejo de días inhabiles, tasas fixing pasadas, entre otras). El objeto `ql.IborIndex` tiene por defecto la definición de las tasas fixing "in advance", es decir, que la tasa que se usará para el pago del cupón se define por completo antes de que empiece el cupón. Es por eso que la curva TIIE28 se considera "Ibor-like"
Se definirá una función que creará todos los "helpers" necesarios para crear la curva. Esta función recibirá, 3 insumos: `tasas`, `tenors` y `curva de descuento`. Los primeros dos insumos ya los tenemos y el tercero se definirá en la sección de Curva de Descuento Mexicana

In [ ]:
def TIIE28Helpers(tasas, tenors, curva_descuento):
    dias_liq = 1 # Dias de liquidación (1)
    cont_dias = ql.Actual360() # Forma de conteo de días 
    frecuencia = ql.EveryFourthWeek # Frecuencia de pagos
    ajuste_fec = ql.Following # Siempre se va al sig día 
    moneda = ql.MXNCurrency() # moneda a usar
    calendario = ql.Mexico() # Calendario de México
    ult_dia_mes = False # Si obliga a terminar al final de mes

    ibor_MXNTIIE = ql.IborIndex(
        'TIIE', # Nombre del indice 
        ql.Period(13), # Se divide el año en 13 periodos
        dias_liq, # Días de liquidación 
        moneda, # Moneda usada
        calendario, 
        ajuste_fec,
        ult_dia_mes, 
        cont_dias)

    descuento = ql.RelinkableYieldTermStructureHandle() # Objeto para usar curvas
    descuento.linkTo(curva_descuento) # le agregamos la curva de descuento

    depositHelpers = [ql.DepositRateHelper(
        ql.QuoteHandle(ql.SimpleQuote(r/100)),
        ql.Period(t, ql.Months),
        dias_liq,
        calendario,
        ajuste_fec,
        ult_dia_mes,
        cont_dias)
        for (r, t) in zip([tasas[0]], [tenors[0]])]

    swapHelpers = [
        ql.SwapRateHelper(
            ql.QuoteHandle(ql.SimpleQuote(r/100)),
            ql.Period(t*4, ql.Weeks),
            calendario,
            frecuencia,
            ajuste_fec,
            cont_dias,
            ibor_MXNTIIE,
            ql.QuoteHandle(),
            ql.Period(),
            descuento)
        for (r, t) in zip(tasas[1:], tenors[1:])]

    helpers = depositHelpers + swapHelpers
    return helpers